## IBM Applied Data Science Capstone Project
#### Onur Başar, 03/23/2021

### 1. Introduction

This project aims to analyze locational data using web scraping and APIs methods corresponds to IBM Data Science Capstone Project. In this project, we will analyze the venues of three big cities, namely Toronto, New York and Berlin. 
Imagine that you just finished your bachelor programm and you want to register a Master programm in one of this 3 big cities. In the matter of education quality, 3 big cities have equal education quality. What you want explore is that the venues of this cities. For example, how many libraries, caffee shops, sport events, concerts etc. are there? And of course, how much money do you need in order to pay your rents and sustain your life. We will compare the venues and costs, and we will try to choose a city based on our properties in the conclusion.

### 2. Data

1. Foursquare: Foursquare is an American technology company which location platform is the foundation of several business and consumer products, including the Foursquare City Guide and Foursquare Swarm apps. The Places API offers real-time access to Foursquare’s global database of rich venue data and user content to power your location-based experiences in your app or website. In the project, we will use Foursquare API in order to retrieve venues of locations.
2. Wikipedia: Wikipedia is a free multilingual wiki-based online encyclopedia edited and maintained by a community of volunteer editors. We will use web scraping on Wikipedia web page in order to get the postal codes or addresses of cities. 
3. Realtor: In order to find out the rent available places and the average rent payment of the cities, we will use Realtor API.